In [ ]:
# Film Corpus 2.0
# Overview: This corpus is an updated version of the Film Corpus 1.0. It contains complete texts for the scripts of 1068 films in txt files, scraped from imsdb.com on Nov, 2015 using scrapy. It also contains 960 film scripts where the dialog in the film has been separated from the scene descriptions.

# The Data: Film scripts are classified by genre,  but one film can be in multiple genres. There are fewer than 1068 separated scripts because we use our own script to automatically separate the dialog and scene descriptions.

# Corpus from: https://nlds.soe.ucsc.edu/fc2

# Only scripts from Action, Comedy, Drama, Thriller have been selected due to size